In [1]:
import plotly.express as px
import pandas as pd
import segyio
import numpy as np

In [ ]:
"""
Алгоритм:
1) куб переделывается в словарь {координаты по инлайнам и крослайнам:[матожидание, дисперсия сейсмотрассы]}
2) 



"""

In [4]:
"""
Тут будем работать с шитыми кубами у которых нет кровли и подошвы и разломов. Поэтому ридер пока мой. (напиши мне, чтобы я скинул сшитые кубы)
Нужно будет потом из датафрейма сделать словарь как в функции
"""
def read_seismic_cube(file_path):
    """
    :param file_path: Путь к файлу
    :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
    а также словарь, где ключ это координата пересечения Inlines и Xlines, а значение это список матожидание и дисперсия сейсмотрассы
    """
    with segyio.open(file_path, 'r') as segyfile:
    	x = list(segyfile.ilines)
    	y = list(segyfile.xlines)
    	z = list(segyfile.samples)
    	cube = segyio.tools.cube(segyfile)
    	M_D = {f"{x[i]}_{y[j]}": [np.mean(cube[i][j][668:1340]), np.var(cube[i][j][668:1340])]  for i in range(len(x)) for j in range(len(y))}
    return x, y, z, M_D

In [18]:
file_path = 'C:/HV/Seismic/datas/shitie_cubs_TWT.segy'
ilines_TWT, xlines_TWT, samples_TWT, M_D_TWT = read_seismic_cube(file_path)

In [57]:
def procent(list1, mid_index):
    """
    Список list1 делим на две части по индексу mid_index и находим процент отличия трендов
    """

    data = np.array(list1)
    
    # Разделение списка на две части
    y1 = data[:mid_index]  # Первая часть
    y2 = data[mid_index:]  # Вторая часть
    x1 = np.arange(1, mid_index + 1)  # Ось x для первой части
    x2 = np.arange(mid_index + 1, len(data) + 1)  # Ось x для второй части
    
    # Вычисление коэффициентов линейной регрессии (y = mx + b)
    m1, b1 = np.polyfit(x1, y1, 1)  # Линия тренда для первой части
    m2, b2 = np.polyfit(x2, y2, 1)  # Линия тренда для второй части
    # Вычисление разницы в наклонах
    slope_difference = abs(m1 - m2)
    average_slope = (abs(m1) + abs(m2)) / 2
    
    # Вычисление процента отличия
    if average_slope != 0:  # Проверка на ноль, чтобы избежать деления на ноль
        percentage_difference = (slope_difference / average_slope) * 100
    else:
        percentage_difference = 0
    
    if percentage_difference <25:
        return 0
    else:
        return percentage_difference

In [97]:
def srednee(list1, mid_index):
    """
    Список list1 делим на две части по индексу mid_index и находим процент отличия средних значений
    """
    # Исходный список данных
    data = np.array(list1)
    
    # Разделение списка на две части
    
    y1 = data[:mid_index]  # Первая часть
    y2 = data[mid_index:]  # Вторая часть
   
    # Вычисление средних значений
    mean1 = np.mean(y1)  # Среднее значение для первой части
    mean2 = np.mean(y2)  # Среднее значение для второй части
    
    # Вычисление разницы в средних значениях
    mean_difference = abs(mean1 - mean2)
    average_mean = (mean1 + mean2) / 2
    
    # Вычисление процента отличия
    if average_mean != 0:  # Проверка на ноль, чтобы избежать деления на ноль
        percentage_difference = (mean_difference / average_mean) * 100
    else:
        percentage_difference = 0
    if percentage_difference <50:
        return 0
    else:
        return percentage_difference


In [105]:
granica_xline = []
for i in range(len(ilines_TWT)):
    line_M = []
    line_D = []
    
    # индексы сшития исходя из линии тренда M и D, исходя из среднего значения M и D 
    index_shit_cub = [0, 0, 0, 0] 
    
    procent_M = []
    procent_D = []
    srednee_M = []
    srednee_D = []
    for j in range(len(xlines_TWT)):
        line_M.append(M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"][0])
        line_D.append(M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"][1])

    df = pd.DataFrame({'Значения_M':line_M, 'Значения_D':line_D})

    df['Экспоненциальное_сглаживание_M'] = df['Значения_M'].ewm(span=40, adjust=False).mean() 
    df['Экспоненциальное_сглаживание_D'] = df['Значения_D'].ewm(span=40, adjust=False).mean()
    
    #оконная функция для нахождения разницы процентов линии тренда
    shag = 10
    for k in range(shag,len(df['Экспоненциальное_сглаживание_M'])-shag,shag):
        procent_M.append(procent(df['Экспоненциальное_сглаживание_M'], k))
        procent_D.append(procent(df['Экспоненциальное_сглаживание_D'], k))
        srednee_M.append(srednee(df['Экспоненциальное_сглаживание_M'], k))
        srednee_D.append(srednee(df['Экспоненциальное_сглаживание_D'], k))
    index_shit_cub[0] = (procent_M.index(max(procent_M))+1) * shag
    index_shit_cub[1] = (procent_D.index(max(procent_D))+1) * shag
    index_shit_cub[2] = (srednee_M.index(max(srednee_M))+1) * shag
    index_shit_cub[3] = (srednee_D.index(max(srednee_D))+1) * shag
    
    filtered_list = [item for item in index_shit_cub if item >= len(df['Экспоненциальное_сглаживание_M'])*0.25 and item <=len(df['Экспоненциальное_сглаживание_M'])*0.75]
    if not filtered_list:
        filtered_list = [0]
        #granica_xline.append(0)
    else:
        granica_xline.append(round(np.mean(filtered_list)))
    #granica_xline.append(index_shit_cub)
    df = df.iloc[0:0] 
    

In [106]:
print(granica_xline)
print(np.mean(granica_xline))
print(len(granica_xline))

[100, 140, 100, 130, 110, 70, 60, 95, 150, 75, 60, 70, 90, 70, 70, 60, 60, 60, 60, 60, 80, 80, 60, 60, 60, 60, 90, 75, 80, 100, 60, 60, 60, 100, 100, 120, 70, 100, 110, 95, 100, 70, 100, 80, 85, 90, 80, 70, 90, 90, 90, 85, 105, 120, 100, 95, 110, 85, 95, 110, 130, 70, 60, 70, 80, 60, 90, 90, 100, 110]
86.0
70


In [100]:
granica_iline = []
for i in range(len(xlines_TWT)):
    line_M = []
    line_D = []
    
    # индексы сшития исходя из линии тренда M и D, исходя из среднего значения M и D 
    index_shit_cub = [0, 0, 0, 0] 
    
    procent_M = []
    procent_D = []
    srednee_M = []
    srednee_D = []
    for j in range(len(ilines_TWT)):
        line_M.append(M_D_TWT[f"{ilines_TWT[j]}_{xlines_TWT[i]}"][0])
        line_D.append(M_D_TWT[f"{ilines_TWT[j]}_{xlines_TWT[i]}"][1])

    df = pd.DataFrame({'Значения_M':line_M, 'Значения_D':line_D})

    df['Экспоненциальное_сглаживание_M'] = df['Значения_M'].ewm(span=40, adjust=False).mean() 
    df['Экспоненциальное_сглаживание_D'] = df['Значения_D'].ewm(span=40, adjust=False).mean()
    
    #оконная функция для нахождения разницы процентов линии тренда
    shag = 10
    for k in range(shag,len(df['Экспоненциальное_сглаживание_M'])-shag,shag):
        procent_M.append(procent(df['Экспоненциальное_сглаживание_M'], k))
        procent_D.append(procent(df['Экспоненциальное_сглаживание_D'], k))
        srednee_M.append(srednee(df['Экспоненциальное_сглаживание_M'], k))
        srednee_D.append(srednee(df['Экспоненциальное_сглаживание_D'], k))
    index_shit_cub[0] = (procent_M.index(max(procent_M))+1) * shag
    index_shit_cub[1] = (procent_D.index(max(procent_D))+1) * shag
    index_shit_cub[2] = (srednee_M.index(max(srednee_M))+1) * shag
    index_shit_cub[3] = (srednee_D.index(max(srednee_D))+1) * shag
    
    filtered_list = [item for item in index_shit_cub if item >= len(df['Экспоненциальное_сглаживание_M'])*0.25 and item <=len(df['Экспоненциальное_сглаживание_M'])*0.75]
    if not filtered_list:
        filtered_list = []
    else:
        granica_iline.append(round(np.mean(filtered_list)))
    
    #granica_iline.append(round(np.mean(index_shit_cub)))
    #granica_iline.append(index_shit_cub)
    df = df.iloc[0:0] 

In [101]:
print(granica_iline)
print(np.mean(granica_iline))

[40, 50, 40, 35, 30, 40, 40, 40, 30, 30, 40, 45, 40, 40, 40, 30, 30, 40, 50, 40, 50, 70, 50, 50, 50, 50, 50, 70, 30, 60, 60, 70, 45, 60, 70, 50, 35, 40, 45, 40, 50, 43, 40, 50, 50, 70, 50, 57, 30, 35, 33, 40, 40, 70, 70, 45, 45, 30, 60, 60, 40, 40, 30, 60, 30, 50, 30, 30, 30, 30, 60, 40, 40, 30, 45, 45, 60, 60, 57, 63, 50, 70, 57, 35, 65, 70, 43, 70, 70, 30, 60, 50, 45, 55, 45, 50, 35, 55, 50, 65, 65, 40, 35, 40, 50, 57, 70, 30, 60, 30, 40, 40, 50, 60, 60, 60, 63, 40, 50, 70, 65, 50, 47, 53, 30, 40, 40, 47, 40, 40, 30, 30, 55, 60, 70, 30, 70, 70, 45, 50, 50, 50, 50, 50, 50, 47, 50, 50, 60, 60, 50, 55, 60]
48.21568627450981
